In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
from datetime import datetime
import csv
import os

In [11]:
# Configurarea WebDriver
options = Options()
options.add_experimental_option("detach", True)

driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

nume_fis='scrapping_booking.csv'

checkin_date = "2025-05-10"
checkout_date = "2025-05-12"
locatie = "Campina"
selected_currency = "RON"
nr_adulti = 2
nr_camere = 1
nr_copiii = 2
limba = "ro"
nr_nopti=datetime.strptime(checkout_date,"%Y-%m-%d")-datetime.strptime(checkin_date,"%Y-%m-%d")

# print((nr_nopti).days
nr_nopti=int((nr_nopti).days)
# print(nr_nopti)
# if nr_nopti > 1:
    #voi imparti pretul la numarul de nopt ca sa aflu pretul per noapte 
# fac un if pentru a imparti pretul la numrarul de nopti daca cumva este mai mult de una

page_url = (f'https://www.booking.com/searchresults.en-us.html?checkin={checkin_date}&checkout={checkout_date}'
            f'&selected_currency={selected_currency}&ss={locatie}'
            f'&ssne={locatie}&ssne_untouched={locatie}&lang={limba}&sb=1&src_elem=sb'
            f'&src=searchresults&dest_type=city&group_adults={nr_adulti}'
            f'&no_rooms={nr_camere}&group_children={nr_copiii}')

driver.get(page_url)

# Așteptare până se încarcă hotelurile
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located(("css selector", "div[data-testid='property-card-container']"))
)
# Colectare date despre hoteluri
hotels = driver.find_elements_by_css_selector("div[data-testid='property-card-container']")

for hotel in hotels:
    try:
        # Numele hotelului
        name_element = hotel.find_element_by_css_selector("div[data-testid='title']")
        name = name_element.text

        # Prețul hotelului
        price_element = hotel.find_elements_by_css_selector("span[data-testid='price-and-discounted-price']")
        price = price_element[0].text # if price_element else "Preț indisponibil"
        pret=float(price.split()[0])
#         print(type(pret))
#         print(pret)

#         nr_nopti=(nr_nopti).days
        if nr_nopti > 1:
            #voi imparti pretul la numarul de nopti ca sa aflu pretul per noapte 
        # fac un if pentru a imparti pretul la numrarul de nopti daca cumva este mai mult de una
            pret_noapte=round((pret/nr_nopti)*1000,2)
        else: 
                  pret_noapte=pret

        #nr de recenzii 
        
        nr_recenzii_element= hotel.find_elements_by_css_selector("div[data-testid='review-score']")
#         nr_recenzie=nr_recenzii_element[0].text if nr_recenzii_element else "Nu sunt recenzii!"
#          if nr_recenzie == 'Scor:'
        if nr_recenzii_element:
            review_text = nr_recenzii_element[0].text  # Ex: "9.4\nSuperb\n911 evaluări"

           
            lines = review_text.split("\n")
            if len(lines) >= 3:
                nr_recenzie = lines[-1].split()[0] 
            else:
                nr_recenzie = 0
        else:
            nr_recenzie =0
          
        
        nr_recenzie=float(nr_recenzie)
        
        #scorul recenziilor
        
        scorul=hotel.find_elements_by_css_selector("div[data-testid='review-score']")
        for element in scorul:
            text = element.text  
    
            scor = re.search(r"\d+,\d+", text)  
            if scor:
                scor_formatat = scor.group().replace(",", ".")
                scor_n = float(scor_formatat) 
                scor_n = scor_n if scor_n else 0

#                 print(scor_n) 


        #nr de stele al hotelului
        
#         nr_stele=hotel.find_elements_by_css_selector("div[data-testid='review-stars']")
#         for s in nr_stele:
#             nr=s.text
        elements= hotel.find_elements_by_css_selector("div[data-testid='rating-squares']")
#         element = soup.find_all('div', {'aria-label': re.compile(r'Număr de stele')})
#         element = element.find_elements_by_css_selector('[aria-label*="Număr de stele"]')
#         scor_eva=soup.find_all('div',{''})
        numar_stele = 0


        for element in elements:
            stele = element.find_elements_by_css_selector("span")
            numar_stele = len(stele) 
#         print(numar_stele)
    
    
    
        # adaugat in excel/csv/sqlserver 
            #-> sa tin cont cand se suprascrie fisierul sa fie append nu delete continut
        
        
        
        
        #analiza datelor optinute
#         print(type(name))
#         print(type(pret_noapte))
#         print(type(numar_stele))
#         print(type(scor_n))
#         print(type(nr_recenzie)) # trebuie sa il transform in int sau float 
#         print(type(locatie))

        date=[name,locatie,pret_noapte,numar_stele,scor_n,nr_recenzie,nr_nopti,checkin_date,checkout_date,nr_camere,nr_adulti,nr_copiii]
        if not os.path.exists(nume_fis):
            with open(nume_fis,mode='w',newline='',encoding='utf-8') as fisier_csv:
                writer=csv.writer(fisier_csv,delimiter=',')
                writer.writerow(['Denumire','Locatie','Pret_noapte','Nr_stele','Scor','Nr_evaluari','Nr_nopti','Check-in','Check-out','Nr_camere','Nr_adulti','Nr_copiii'])

        with open(nume_fis,mode='a',newline='',encoding='utf-8') as fisier_csv:
            writer=csv.writer(fisier_csv,delimiter=',')
            writer.writerow(date)
#         print(f"Datele au fost adăugate cu succes în fișierul: {nume_fis}")
#             for hotel in hotels:
#                 date=[name,locatie,pret_noapte,numar_stele,scor_n,nr_recenzie,nr_nopti,checkin_date,checkout_date,nr_camere,nr_adulti,nr_copiii]
          

 
    #toate datele sunt de acelasi tip 

    

        print(f"{name} | Pret_noapte: {pret_noapte} | Nr_stele: {numar_stele} | Scor: {scor_n} | Nr_evaluari: {nr_recenzie} | Locatie: {locatie}")
    except Exception as e:
        print(f"Eroare la extragere: {e}")
# for dat in date:
#     writer.writerow(date)
# print(f"Datele au fost adăugate cu succes în fișierul: {nume_fis}")
# if not os.path.exists(nume_fis):
#     with open(nume_fis,mode='w',newline='',encoding='utf-8') as fisier_csv:
#         writer=csv.writer(fisier_csv,delimiter=',')
#         writer.writerow(['Denumire','Locatie','Pret_noapte','Nr_stele','Scor','Nr_evaluari','Nr_nopti','Check-in','Check-out','Nr_camere','Nr_adulti','Nr_copiii'])

# with open(nume_fis,mode='a',newline='',encoding='utf-8') as fisier_csv:
#     writer=csv.writer(fisier_csv,delimiter=',')
#     for hotel in hotels:
#          date=[name,locatie,pret_noapte,numar_stele,scor_n,nr_recenzie,nr_nopti,checkin_date,checkout_date,nr_camere,nr_adulti,nr_copiii]
#         for dat in date:
#             writer.writerow(date)
# print(f"Datele au fost adăugate cu succes în fișierul: {nume_fis}")

# Închidem browserul manual după rulare
input("Apasă ENTER pentru a închide browserul...")
driver.quit()


Hotel Premium Campinas | Pret_noapte: 490000.0 | Nr_stele: 0 | Scor: 8.6 | Nr_evaluari: 1.39 | Locatie: Campina
Hotel Vila Rica Campinas | Pret_noapte: 281500.0 | Nr_stele: 0 | Scor: 8.0 | Nr_evaluari: 1.965 | Locatie: Campina
Hotel PrimeInn | Pret_noapte: 271500.0 | Nr_stele: 0 | Scor: 8.2 | Nr_evaluari: 2.163 | Locatie: Campina
Hotel Casablanca | Pret_noapte: 230500.0 | Nr_stele: 0 | Scor: 8.7 | Nr_evaluari: 1.796 | Locatie: Campina
Hotel Esplanada | Pret_noapte: 265500.0 | Nr_stele: 0 | Scor: 7.1 | Nr_evaluari: 1.397 | Locatie: Campina
Pousada Santa Genebra | Pret_noapte: 145000.0 | Nr_stele: 0 | Scor: 9.4 | Nr_evaluari: 224.0 | Locatie: Campina
Hotel Village Campinas | Pret_noapte: 273500.0 | Nr_stele: 0 | Scor: 8.7 | Nr_evaluari: 789.0 | Locatie: Campina
Pousada Happy Inn | Pret_noapte: 233500.0 | Nr_stele: 0 | Scor: 9.2 | Nr_evaluari: 677.0 | Locatie: Campina
K1 RESIDENCE TAQUARAL com garagem incluída | Pret_noapte: 360000.0 | Nr_stele: 3 | Scor: 8.8 | Nr_evaluari: 551.0 | Locati